In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

Вопрос 1.

Задача: Прогнозирование вероятности покупки товара в интернет-магазине

Студент B посетил интернет-магазин и просмотрел товары из категории "Электроника", сумма его предыдущей покупки составила 1500 рублей, а длительность сессии - 20 минут.

У нас есть обученная модель логистической регрессии с коэффициентами:

сумма покупки - 0.002

категория "Электроника" - 0.5

длительность сессии - 0.1

свободный коэффициент - -5

Получите вероятность покупки студентом B в интернет-магазине и укажите ответ, округлив его до второго знака после точки.

$P(y=1) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3)}}$

In [3]:
x = -5 + (0.002 * 1500) + (0.5 * 1) + (0.1 * 20)
p = np.round(1 / (1 + np.exp(-x)), 2)
print(p)

0.62


---

Вопрос 2

Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Имеется датасет для предсказания оценок студентов. Но мы возьмём немного измененную версию данных, скачать датасет.

Сделайте разбиение на обучение и тест с соотношением 80/20 и random_state=10.

Целевой признак grade: 0 - оценка ниже среднего, 1 - оценка выше среднего.

Обучите логистическую регрессию из sklearn на базовых параметрах для решения поставленной задачи и укажите вес, который получился после обучения модели, при признаке studytime, округлив его до второго знака после точки.

In [4]:
df = pd.read_csv('./data/students.csv')
df.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,grade
0,18,4,4,2,2,0,4,3,4,1,1,3,4,1
1,17,1,1,1,2,0,5,3,3,1,1,3,2,1
2,15,1,1,1,2,0,4,3,2,2,3,3,6,1
3,15,4,2,1,3,0,3,2,2,1,1,5,0,2
4,16,3,3,1,2,0,4,3,2,1,2,5,0,1


In [5]:
X = df.drop(columns=["grade"])
y = df["grade"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)

model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)

result = dict(zip(X.columns, np.round(model.coef_[0], 2)))
result_df = pd.DataFrame(result.items(), columns=['Feature', 'Coefficient'])
display(result_df)

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Feature,Coefficient
0,age,0.18
1,Medu,0.46
2,Fedu,0.02
3,traveltime,-0.14
4,studytime,0.38
5,failures,-1.33
6,famrel,0.00
7,freetime,0.05
8,goout,-0.02
9,Dalc,-0.67


---
Вопрос 3.

Продолжайте работать с данными с предыдущего задания.

Вероятней всего (если у вас не отключены предупреждения) Python выдал 

ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT., 

такое встречается, если в данных есть признаки разных масштабов. Давайте исправим это с помощью масштабирования данных через стандартизацию.

Обучите модель StandardScaler на тренировочной выборке.

Преобразуйте тренировочные и тестовые данных с помощью стандартизации

Укажите, какие диапазоны получились у признака age на тестовой выборке - значение min и max. 

In [6]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

answer_df = pd.DataFrame(dict(zip(X.columns, X_test_sc)))
answer_df["age"].describe().round(2)

count    13.00
mean     -0.12
std       0.88
min      -1.30
25%      -0.63
50%      -0.29
75%       0.10
max       1.57
Name: age, dtype: float64

---
Вопрос 4.

Продолжайте работать с масштабированными данными с предыдущего задания.

Обучите модель логистической регрессии с базовыми параметрами на стандартизованных данных и посчитайте метрику accuracy на тренировочной и тестовой выборках и укажите их, округлив до второго знака после точки.

In [7]:
model = LogisticRegression()
model.fit(X_train_sc, y_train)

y_pred_test = model.predict(X_test_sc)
y_pred_train = model.predict(X_train_sc)

acc_test = np.round(accuracy_score(y_test, y_pred_test), 2)
acc_train = np.round(accuracy_score(y_train, y_pred_train), 2)
print(acc_train, acc_test)

0.76 0.78


---
Вопрос 5.

Продолжайте работать с данными с предыдущего задания.

Возьмите НЕ масштабированные данные, т.к. сейчас будем их усложнять для более качественной модели.

Для удобства можете пользоваться pipeline из sklearn, в котором будут:

Модель полиномизации признаков до 4 степени PolynomialFeatures

Модель масштабирования данных StandardScaler

Логистическая регрессия БЕЗ регуляризации

По умолчанию в логистической регрессии в sklearn есть L2 регуляризация, чтобы её отключить, нужно поменять параметр penalty.

Обучите пайплайн на тренировочной выборке и посчитайте метрику accuracy на тренировочной и тестовой выборках и укажите их, округлив до второго знака после точки.

In [8]:
X = df.drop(columns=["grade"])
y = df["grade"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)

model_pipe = Pipeline([
    ("poly", PolynomialFeatures(degree=4)),
    ("scale", StandardScaler()),
    ("log_reg", LogisticRegression(penalty=None))
])

model_pipe.fit(X_train, y_train)

y_pred_test = model_pipe.predict(X_test)
y_pred_train = model_pipe.predict(X_train)

acc_test = np.round(accuracy_score(y_test, y_pred_test), 2)
acc_train = np.round(accuracy_score(y_train, y_pred_train), 2)
print(acc_train, acc_test)

0.95 0.68


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---
Вопрос 6

Продолжайте работать с данными с предыдущего задания.

Снова вероятней всего (если у вас не отключены предупреждения) Python выдал ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT., хотя масштабирование данных сделано, так же эту ошибку можно избежать, если увеличить количество итераций в логистической регрессии, увеличьте параметр max_iter до 600.

Так же метрика на тесте стала меньше, потому что модель переобучается, нужно и это исправить с помощью регуляризации.

Для удобства можете пользоваться pipeline из sklearn, в котором будут:

Модель полиномизации признаков до 4 степени PolynomialFeatures

Модель масштабирования данных StandardScaler

Логистическая регрессия с L2 регуляризацией

Сделайте перебор параметра C в логистической регрессии, который из себя представляет 1/коэффициент регуляризации. Значения C перебирайте из следующего списка: [0.0001, 0.001, 0.01, 0.1, 1].

Укажите, самую лучшую метрику на тесте, округлив её до второго знака после точки.

In [9]:
param_grid = {"log_reg__C": [0.0001, 0.001, 0.01, 0.1, 1]}

model_pipe_2 = Pipeline([
    ("poly", PolynomialFeatures(degree=4)),
    ("scale", StandardScaler()),
    ("log_reg", LogisticRegression(penalty='l2', max_iter=700))
])

gs_cv = GridSearchCV(
    estimator=model_pipe_2,
    param_grid=param_grid,
    n_jobs=-1,
    scoring='accuracy'
)

gs_cv.fit(X_train, y_train)

gs_cv.best_params_

{'log_reg__C': 0.0001}

In [10]:
y_pred_test = gs_cv.predict(X_test)
test_accuracy = np.round(accuracy_score(y_test, y_pred_test), 2)
print("Лучшая метрика (accuracy) на тестовой выборке:", test_accuracy)

Лучшая метрика (accuracy) на тестовой выборке: 0.81


---
Вопрос 7.

Имеется датасет для чаевых для таксиста . Но мы возьмём немного измененную версию данных. Скачать датасет.

Сделайте разбиение на обучение и тест с соотношением 80/20 и random_state=10.

Целевой признак tip_class.

Обучите логистическую регрессию из sklearn с параметром max_iter=400 для решения поставленной задачи.

Изучите масштаб в данных и если это требуется примените StandardScaler.

Укажите метрики accuracy на тренировочной и тестовой выборках, округлив их до второго знака после точки. 

In [11]:
df_2 = pd.read_csv("./data/trip.csv")
df_2.head()

,trip_duration,distance_traveled,num_of_passengers,fare,miscellaneous_fees,total_fare,surge_applied,tip_class
0,748.0,2.75,1.0,75.00,6.300,105.300,0.0,over med
1,1187.0,3.43,1.0,105.00,13.200,142.200,0.0,over med
2,329.0,2.09,1.0,45.00,13.200,70.200,0.0,under med
3,355.0,1.74,1.0,45.00,26.625,71.625,1.0,low
4,1288.0,5.21,1.0,116.25,26.325,178.575,1.0,high


In [12]:
df_2.isna().sum()

trip_duration         0
distance_traveled     0
num_of_passengers     0
fare                  0
miscellaneous_fees    1
total_fare            1
surge_applied         1
tip_class             1
dtype: int64

In [13]:
df_2.dropna(inplace=True)

In [ ]:
X = df_2.drop(columns=["tip_class"])
y = df_2["tip_class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)

model_3 = Pipeline([
    ("scale", StandardScaler()),
    ("log_reg", LogisticRegression(max_iter=400, n_jobs=-1))
])

model_3.fit(X_train, y_train)

y_train_pred = model_3.predict(X_train)
y_test_pred = model_3.predict(X_test)

acc_test = np.round(accuracy_score(y_test, y_test_pred), 2)
acc_train = np.round(accuracy_score(y_train, y_train_pred), 2)
print(acc_train, acc_test)

0.58 0.56


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---
Вопрос 8.

Продолжайте работать с датасетом предыдущего задания.

Посчитайте, сколько получилось ошибочных предсказаний в процентном соотношении по каждому классу на тестовой выборке и укажите в ответе, округлив проценты до второго знака после точки.

Пример ответа:

low: 5.10% ошибочных предсказаний under med: 15.26% ошибочных предсказаний over med: 18.93% ошибочных предсказаний high: 6.24% ошибочных предсказаний



In [19]:
# Создадим DataFrame из истинных меток и предсказаний
test_results = pd.DataFrame({
    'true': y_test,  # Убедитесь, что это одномерный массив
    'predicted': y_test_pred
})

# Получаем список уникальных классов
classes = test_results['true'].unique()

# Инициализируем словарь для хранения процентных значений
error_percentages = {}

# Подсчитываем ошибочные предсказания для каждого класса
for cls in classes:
    total_count = (test_results['true'] == cls).sum()  # Общее количество экземпляров в классе
    incorrect_count = ((test_results['predicted'] != cls) & (test_results['true'] == cls)).sum()  # Неправильные предсказания
    error_percentage = (incorrect_count / total_count) * 100 if total_count > 0 else 0  # Это предотвратит деление на ноль
    error_percentages[cls] = round(error_percentage, 2)  # Округляем до двух знаков

# Форматируем вывод
for cls, error in error_percentages.items():
    print(f"{cls}: {error:.2f}% ошибочных предсказаний")

low: 8.19% ошибочных предсказаний
under med: 82.07% ошибочных предсказаний
high: 10.14% ошибочных предсказаний
over med: 69.28% ошибочных предсказаний
